In [ ]:
import privacy_evaluator.models.torch.dcti.dcti as torch_dcti 
from privacy_evaluator.datasets.cifar10 import CIFAR10
from privacy_evaluator.classifiers.classifier import Classifier
from privacy_evaluator.attacks.membership_inference.black_box import MembershipInferenceBlackBoxAttack
from privacy_evaluator.attacks.membership_inference.black_box_rule_based import MembershipInferenceBlackBoxRuleBasedAttack
from privacy_evaluator.attacks.membership_inference.label_only_decision_boundary import MembershipInferenceLabelOnlyDecisionBoundaryAttack

#  Membership Inference Attack Examples
### Prepare datasets and Pytorch target model

In [ ]:
# CIFAR datasets
x_train, y_train, x_test, y_test = CIFAR10.numpy()

# Wrapped Pytorch target model
target_model = Classifier(torch_dcti.load_dcti(), nb_classes=CIFAR10.N_CLASSES, input_shape=CIFAR10.INPUT_SHAPE)

## Membership Inference Black Box Attack
### Prepare attack and attack

In [ ]:
attack = MembershipInferenceBlackBoxAttack(target_model, x_train, y_train, x_test, y_test)
attack.attack()

### Create model cart info

In [ ]:
attack.model_card_info()

## Membership Inference Black Box Rule Based Attack
### Prepare attack and attack

In [ ]:
attack = MembershipInferenceBlackBoxRuleBasedAttack(target_model, x_train, y_train, x_test, y_test)
attack.attack()

### Create model cart info

In [ ]:
attack.model_card_info()

## Membership Inference Label Only Decision Boundary Attack
### Prepare attack and attack

In [ ]:
attack = MembershipInferenceLabelOnlyDecisionBoundaryAttack(target_model, x_train[:1], y_train[:1], x_test[:1], y_test[:1])
attack.attack(max_iter=1, max_eval=1, init_eval=1)

### Create model cart info

In [ ]:
attack.model_card_info()